In [15]:
import pandas as pd
import numpy as np
import pandas as pd
from docxtpl import DocxTemplate,  InlineImage
from docx.shared import Mm
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date
import warnings
warnings.filterwarnings('ignore')
id=204
# id=192
today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
cotizacion=pd.read_sql("select * from quotations where id ="+str(id),cnx)
cart_products=pd.read_sql("select * from cart_products where quotation_id ="+str(id),cnx)

cliente=pd.read_sql("""select * from customers where customers.id="""+str(cotizacion['customer_id'].values[0]),cnx)
user=pd.read_sql("""select * from users where users.id="""+str(cotizacion['user_id'].values[0]),cnx)
productos=pd.read_sql("""select * from cart_products where cart_products.quotation_id="""+str(cotizacion['id'].values[0]),cnx)
questionario=pd.read_sql("""select * from questionaries where questionaries.quotation_id="""+str(cotizacion['id'].values[0]),cnx)
if(len(questionario)==0):
    questionario=pd.read_sql("""select * from questionaries limit 1""",cnx)

import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")

from tablas_dict import tablas
from tablas_dict import redact
from tablas_dict import extras
from tablas_dict import ref
aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

print('producots en carrito ',len(cart_products))
products=pd.DataFrame()
#iterar sobre tablas
for i in tablas:
    
    #buscar en la base de datos todos los productos de esta tabla
    #pertenecientes a la cotizacion pedida por el usuario.
    p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
    p=p.assign(tabla=i)
    cart_reference=[]
    if(len(p)>0):
        cart_reference=cart_products.loc[cart_products['id']==p.cart_id.values[0]]
    if(('cost' not in p.columns)&(len(p)>0)&(len(cart_reference)>0)):
        if('caliber' not in p.columns):
             #esto es en especifico por un caso en que todas kas piezas son cal 14
             p=p.assign(caliber='14')
        try:
            p['caliber']=p['caliber'].str.replace('-','')
        except:
            print(' ')
        costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
        # if('total_kg' in p.columns):
        #     p=p.assign(cost=costo*p.total_kg)
        # if('total_weight' in p.columns):
        #     p=p.assign(cost=costo*p.total_weight)
        # if('weight_kg' in p.columns):
         
        #     p=p.assign(cost=costo*p.weight_kg)
        # if('weight' in p.columns):
          
        #     p=p.assign(cost=costo*p.weight)
        # if('long' in p.columns):
           
        #     p=p.assign(cost=costo*p.long)
    # if(len(p)>0)&(len(cart_reference)>0):
    if(len(p)>0):
        if(len(cart_reference)>0):
            print('ha sido encontrada una instancia de ',p)
            p=p.assign(cost_unit=cart_reference['unit_price'].values[0])
            p=p.assign(cost_total=cart_reference['total_price'].values[0])
            p=p.assign(cantidad=cart_reference['amount'].values[0])
        else:
            # p=p.assign(cost_unit=cart_reference['unit_price'].values[0])
            # p=p.assign(cost_total=cart_reference['total_price'].values[0])
            p=p.assign(cantidad=p['amount'].values[0])
        
    
    products=pd.concat([products,p],ignore_index=True)
# print(products)
cols_to_fill_str=['description','protector','model','sku']
products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
cols_kg=['weight','total_kg','total_weight','weight_kg']
cols_m2=['m2','total_m2']
largo_cols=['long','length','length_meters','frame_background',
       'length_dimension', 'dimensions']
ancho_cols=['uncut_front',  'uncut_background',
       'depth']
products[cols_kg+cols_m2+largo_cols+ancho_cols]=products[cols_kg+cols_m2+largo_cols+ancho_cols].fillna(0)
price_cols=['price','total_price','import','unit_price']
products[price_cols] =products[price_cols].fillna(0)    
pricelist_protectors=pd.read_sql('select * from price_list_protectors',cnx)
quotation_protectors=pd.read_sql('select quotation_protectors.*, protectors.sku from quotation_protectors  inner join protectors on protectors.protector=quotation_protectors.protector where quotation_id ='+str(id),cnx)
quotation_shlf=pd.read_sql('select * from selective_heavy_load_frames where quotation_id ='+str(id),cnx)
materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
materials['type']=materials['type'].fillna('')

if(cotizacion['type'].values[0]=='DOBLE PROFUNDIDAD'):
    doc= DocxTemplate("plantilla_d.docx")
else:
    if(cotizacion['type'].values[0]=='DRIVE IN'):
        doc= DocxTemplate("plantilla_drivein.docx")
    else:
        doc = DocxTemplate("plantilla.docx")



instalacion_tables=['quotation_installs','quotation_uninstalls']
productos=[]
print(len(products),products.columns)
for i in range(len(products)):
    this_color=' '
    seccion=None
    carga=None
    altura=0
    ancho=0
    # print(products['tabla'].values[i],products['cantidad'].values[i],products['cost_unit'].values[i])
    if(products['tabla'].values[i] not in instalacion_tables):
        if(products['tabla'].values[i]=='selective_heavy_load_frames'):
            this_color='Azul'
            seccion=questionario['section'].values[0]
        if('joist' in products['tabla'].values[i]):
            this_color='Anaranjado'
        if(('brazo' in products['tabla'].values[i])|('arrios' in products['tabla'].values[i])):
            this_color='Anaranjado'
            carga='{0:.2f}'.format(products['weight_kg'].values[i])
        
        if(products['cantidad'].values[i]>0):
            print('        SI----')
            productos.append({'nombre':redact[products['tabla'].values[i]],
                        'extra':extras[products['tabla'].values[i]],                          
                        'ref':ref[products['tabla'].values[i]],
                        'precio':products[price_cols].sum(axis=1).values[i],
                        'cantidad':products['cantidad'].values[i],
                        'color': this_color,
                        'largo': products[largo_cols].sum(axis=1).values[i],
                        'carga': carga,
                        'altura': products[largo_cols].sum(axis=1).values[i],
                        'ancho': products[ancho_cols].sum(axis=1).values[i],
                        'depth': products['depth'].values[i],
                        'model': products['model'].values[i],
                        'seccion':seccion})

print('X-X-X-X-X-X-X calculado el total',products[price_cols]) 

precio_total=cart_products['total_price'].sum()
print(precio_total)
kilos_totales=products[cols_kg].sum(axis=1).sum()
fletes_tables=['packagings','quotation_travel_assignments']
instalacion_tables=['quotation_installs','quotation_uninstalls']
precios=products[price_cols+['tabla','print']]
costo_flete=precios.loc[precios['tabla'].isin(fletes_tables)].sum(axis=1,numeric_only=True).sum()
costo_instalacion=precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='Sí')].sum(axis=1,numeric_only=True).sum()
costo_instalacion_incluida=precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='In')].sum(axis=1,numeric_only=True).sum()

if(costo_instalacion>0):
    print('la instalacion se desglosa')
    print(precios.loc[(precios['tabla'].isin(instalacion_tables))&(precios['print']=='Sí')])
    des_inst=1
else:
    des_inst=0
text=str(questionario['ndib'].values[0])+','
dibujos=[]
while(',' in text):
    print('entra iteracion')
    myindex=text.index(',')
    print(myindex)
    dibujos.append(text[0:myindex])
    text=text[myindex+1:]

if(len(dibujos)>0):
    primer_dibujo=dibujos[0]
else:
    primer_dibujo=' '
if(cotizacion['img'].values[0]):
    photo=InlineImage(doc,'storage/'+cotizacion['img'].values[0],width=Mm(50))
else:
    photo=''
context={
    'cliente':cliente['customer'].values[0],
    'direccion':cliente['address'].values[0]+' '+cliente['outdoor'].values[0]+', '+cliente['city'].values[0]+' '+cliente['suburb'].values[0]+' '+cliente['state'].values[0]+', cp: '+str(cliente['zip_code'].values[0]),
    'folio': cotizacion['invoice'].values[0],
    'fecha': today,
    'asesor': user['name'].values[0],
    'mayus_type':cotizacion['type'].values[0],
    'type':cotizacion['type'].values[0].capitalize(),
    'productos': productos,
    'precio_total': '{:2,.2f}'.format(precio_total),
    'kilos_totales': '{0:.2f}'.format(kilos_totales),
    'costo_flete':'{0:.2f}'.format(costo_flete),
    'costo_instalacion':'{:2,.2f}'.format(costo_instalacion),
    'costo_instalacion_incluida':'{:2,.2f}'.format(costo_instalacion_incluida),
    'costo_selectivo':'{:2,.2f}'.format(precio_total - costo_flete -costo_instalacion),
    'estado': cliente['state'].values[0],
    'a5': questionario['a5'].values[0], #que productos e almacena
   #reativos de nivel
    'a8': questionario['a8'].values[0],#frente
    'a9': questionario['a9'].values[0],#fondo
    'a10': questionario['a10'].values[0],#alto
    'a11': questionario['a11'].values[0],#peso
  # dimensiones de la tarima
    'a18': questionario['a18'].values[0],#frente
    'a19': questionario['a19'].values[0],#fondo
    'a20': questionario['a20'].values[0],#alto
    'a21': questionario['a21'].values[0],#peso
  #ambiente
    'a25': questionario['a25'].values[0], #temperatura
    'a26': questionario['a26'].values[0], #inflamable
    'a27': questionario['a27'].values[0], #explosivo
    'a28': questionario['a28'].values[0], #corrosivo
#Reactivos extra
#     'ndib':  str(questionario['ndib'].values[0]).replace(',',"""
# """), #numero de dibujos
'ndib':  str(questionario['ndib'].values[0]).replace(',','O'),
'primer_dibujo':primer_dibujo ,
'photo':photo,
    'npos':  questionario['npos'].values[0], #numero de posiciones
    'vigas':  questionario['vigas'].values[0], #vigas
    'tiempo':  questionario['tiempo'].values[0], #tiempo de entrega
    'dibujos': dibujos,
    'des_inst': des_inst
    }
doc.render(context) 
doc.save("storage/Cotizacion"+str(id)+".docx")


producots en carrito  0
1 Index(['id', 'quotation_id', 'amount', 'type', 'depth', 'developing', 'long',
       'caliber', 'kg_m2', 'weight', 'm2', 'connector', 'sku', 'unit_price',
       'total_price', 'created_at', 'updated_at', 'cart_id', 'tabla', 'length',
       'camber', 'height', 'model', 'total_load_kg', 'total_poles',
       'total_crossbars', 'total_diagonals', 'total_plates', 'total_kg',
       'total_m2', 'skate', 'loading_capacity', 'type_joist', 'length_meters',
       'weight_kg', 'use', 'destination', 'front', 'color', 'background',
       'cost', 'dimensions', 'joist_type', 'price', 'state', 'unit', 'import',
       'frame_background', 'length_dimension', 'total_weight', 'price_unit',
       'description', 'system', 'f_total', 'print', 'breakdown_install',
       'protector', 'costo_sn_factor', 'dias', 'operarios',
       'breakdown_uninstall', 'cantidad', 'uncut_front', 'cut_front',
       'uncut_background', 'cut_background', 'desarrollo', 'piezas_nec',
       'piece

In [2]:

redact={
'double_deep_crossbars' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2.5',
'double_deep_floors' : 'SUELOS DE DOBLE PROFUNDIDAD',
'double_deep_floor_reinforcements' : 'REFUERZOS PARA EL SUELO DE DOBLE PROFUNDIDAD',
'double_deep_heavy_load_frames' : 'BASTIDORES DE CARGA PESADA DE DOBLE PROFUNDIDAD',
'double_deep_joist_box25s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2.5',
'double_deep_joist_box25_caliber14s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2.5 CALIBRE 14',
'double_deep_joist_box2s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2',
'double_deep_joist_box2_caliber14s' : 'VIGA TIPO CAJA DE DOBLE PROFUNDIDAD DE 2 CALIBRE 14',
'double_deep_joist_c2_s' : 'VIGA DE DOBLE PROFUNDIAD TIPO C2',
'double_deep_joist_chairs' : 'VIGA DE DOBLE PROFUNDIAD TIPO SILLA',
'double_deep_joist_l25_caliber14s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2.5 CALIBRE 14',
'double_deep_joist_l25_s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2.5 ',
'double_deep_joist_l2_caliber14s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2 CALIBRE 14',
'double_deep_joist_l2_s' : 'VIGA TIPO L DE DOBLE PROFUNDIDAD DE 2 ',
'double_deep_joist_lrs' : 'VIGA DE DOBLE PROFUNDIAD TIPO LR',
'double_deep_joist_structurals' : 'VIGA DE DOBLE PROFUNDIAD TIPO ESTRUCTURAL',
'double_deep_miniature_frames' : 'MARCOS EN MINIATURA DE DOBLE PROFUNDIDAD',
'double_deep_spacers' : 'ESPACIADOR DE DOBLE PROFUNDIDAD',
'double_deep_structural_frames' : 'MARCOS ESTRUCTURALES DE DOBLE PROFUNDIDAD',
'freights' : 'FLETES',
'grills' : 'Parrilla Métalica',
'installations' : 'INSTALACION',
'packagings' : 'FLETE',
'quot25_j_galvanized_panels' : 'Panel metálico Galvanizado modelo ',
'quot25_j_painted_panels' : 'Panel metálico Pintado modelo',
'quot2_j_galvanized_panels' : 'Panel metálico Galvanizado modelo',
'quot2_j_painted_panels' : 'Panel metálico Pintado modelo',
'quotation_administratives' : 'ADMINISTRATIVO',
'quotation_installs' : 'INSTALACION',
'quotation_protectors' : 'Protector ',

'quotation_escuadras' : 'ESCUADRA DE SOPORTE',
'quotation_gangplank_angles' : 'ANGULO RANURADO',
'quotation_respaldos':'RESPALDO ',
'quotation_specials' : 'ESPECIAL',
'quotation_travel_assignments' : 'VIATICOS ',
'quotation_uninstalls' : 'DESINTALACION',
'quot_chair_j_galvanized_panels' : 'Panel métalico galvanizado tipo silla',
'quot_chair_j_painted_panels' : 'Panel Métalico Pintado tipo silla',
'selective_crossbars' : 'Crosbaar metálico galvanizado, modelo ',
'selective_floors' : 'Piso Metálico modelo ',
'selective_floor_reinforcements' : 'Refuerzos para Piso Métalico modelo ',
'selective_heavy_load_frames' : 'Marco de Rack Pesado, con poste ',
'selective_joist_box25s' : 'Viga tipo caja de 2.5 modelo ',
'selective_joist_box25_caliber14s' : 'Viga tipo caja de 2.5 calibre 14 modelo',
'selective_joist_box2s' : 'Viga tipo caja de 2.0 modelo',
'selective_joist_box2_caliber14s' : 'viga tipo caja de 2.0 calibre14',
'selective_joist_c2_s' : 'Viga tipo C2 modelo',
'selective_joist_chairs' : 'Viga tipo silla modelo',
'selective_joist_l25_caliber14s' : 'Viga tipo l25 calibre 14 modelo',
'selective_joist_l25_s' : 'Viga tipo L25 modelo',
'selective_joist_l2_caliber14s' : 'Viga tipo L2 calibre14 modelo',
'selective_joist_l2_s' : 'Viga tipo L2 modelo',
'selective_joist_lrs' : 'Viga tipo LR modelo',
'selective_joist_structurals' : 'Viga tipo estructural modelo ',
'selective_miniature_frames' : 'MARCOS EN MINIATURA',
'selective_spacers' : 'Distanciador modelo',
'selective_structural_frames' : 'Marco de Rack Estructu ral con poste ',
'uninstalls' : 'DESINSTALACION',
'wood' : 'Panel de Madera',

'quotation_drive_in_soportes' : 'Soportes para tarimas',
'quotation_drive_in_guias' : 'Guias para montacargas',
'quotation_drive_in_arriostrados' : 'Arriostrados tipo "L" modelo',
'quotation_drive_in_brazos' : 'Brazos para drive in modelo',
'quotation_galletas':'Galletas'
}


In [17]:
for i in redact :
    
    # print("alter table "+str(i)+" add column cart_id int;")
    print("delete from "+str(i)+" ;")

delete from double_deep_crossbars ;
delete from double_deep_floors ;
delete from double_deep_floor_reinforcements ;
delete from double_deep_heavy_load_frames ;
delete from double_deep_joist_box25s ;
delete from double_deep_joist_box25_caliber14s ;
delete from double_deep_joist_box2s ;
delete from double_deep_joist_box2_caliber14s ;
delete from double_deep_joist_c2_s ;
delete from double_deep_joist_chairs ;
delete from double_deep_joist_l25_caliber14s ;
delete from double_deep_joist_l25_s ;
delete from double_deep_joist_l2_caliber14s ;
delete from double_deep_joist_l2_s ;
delete from double_deep_joist_lrs ;
delete from double_deep_joist_structurals ;
delete from double_deep_miniature_frames ;
delete from double_deep_spacers ;
delete from double_deep_structural_frames ;
delete from freights ;
delete from grills ;
delete from installations ;
delete from packagings ;
delete from quot25_j_galvanized_panels ;
delete from quot25_j_painted_panels ;
delete from quot2_j_galvanized_panels ;
delet

In [8]:
factores.loc[factores['caliber']=='14']

,id,description,caliber,type,system,piece,sku,cost,steel,f_vta,f_desp,f_emb,f_desc,f_total,created_at,updated_at
0,1,LAMINA NEGRA RC,14,Negra,ESTANTERIA,ANGULO,None,30.0,1,2.3,2.0,1.015,0.94,4.915,2023-03-17 00:00:00,2024-07-11 03:30:32
10,11,LAMINA GALVANIZADA RC,14,Galvanizada,SELECTIVO,MARCO,None,25.0,1,2.3,2.0,1.015,0.95,4.915,2023-03-17 00:00:00,2024-07-11 06:10:25
15,16,LAMINA NEGRA RC,14,Negra,SELECTIVO,VIGA,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
22,23,LAMINA NEGRA RC,14,Negra,MINI RACK,MARCO,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
25,26,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,CROSS BAR,None,25.0,1,2.0,2.0,1.015,0.94,4.319,2023-03-17 00:00:00,2024-07-20 05:56:41
30,31,LAMINA NEGRA RC,14,Negra,ACCESORIOS,PANELES,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
31,32,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,PANELES,None,25.0,1,2.0,2.0,1.015,0.94,4.319,2023-03-17 00:00:00,2024-07-11 06:10:25
38,39,LAMINA NEGRA RC,14,Negra,ACCESORIOS,PISO,None,30.0,1,2.3,2.0,1.015,0.94,4.967,2023-03-17 00:00:00,2024-07-11 03:30:32
39,40,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,PISO,None,25.0,1,2.0,2.0,1.015,0.94,4.319,2023-03-17 00:00:00,2024-07-11 06:10:25
40,41,LAMINA GALVANIZADA RC,14,Galvanizada,ACCESORIOS,CALZAS,TC0000000,25.0,1,3.0,2.0,1.015,0.95,4.915,2023-03-17 00:00:00,2024-07-11 06:10:25


In [10]:
for i in range(len(loading)):
    cursor.execute(query,tuple([loading['calibre'].values[i],
                                loading['capacidad'].values[i],
                                loading['longitud'].values[i],
                                loading['peralte'].values[i],
                                   ]))
cnx.commit()

In [1]:
'SELECTIVO'

'Selectivo'

In [11]:
lr=pd.read_excel('tablas_lr.xlsx')

In [16]:
lr=lr.astype(str)

In [18]:
lr.columns

Index(['largo', 'sku', 'peso', 'm2', 'precio', 'peralte', 'calibre'], dtype='object')

In [24]:
query='INSERT INTO type_l_r_joists(length,sku,weight,m2,price,camber,caliber) values(%s,%s,%s,%s,%s,%s,%s)'

In [26]:
for i in range(len(lr)):
    cursor.execute(query,tuple([lr['largo'].values[i],
                                lr['sku'].values[i],
                                lr['peso'].values[i],
                                lr['m2'].values[i],
                                lr['precio'].values[i],   
                                lr['peralte'].values[i],
                                lr['calibre'].values[i],
                                   ]))
cnx.commit()

In [56]:
data=pd.read_excel('entpint.xlsx',sheet_name='ENTREPAÑO C-20',skiprows=[0,2])

In [57]:
data.columns

Index(['DFONDO M', 'DFRENTE M', 'VFONDO M', 'VLARGO M', 'SKU', 'CALIBRE',
       'KG / M2', 'PESO ', 'M2 ', 'PRECIO VENTA', 'CAPACIDAD DE CARGA (KG)',
       'CON 1 REFUERZO', 'CON 2 REFUERZOS'],
      dtype='object')

In [59]:
query='INSERT INTO estanteria_entrepanios(development_deep,development_front,deep,length,sku,caliber,kgm2,weight,m2,loading_capacity,reforcement1,reforcement2,type) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [65]:
for caliber in ['20','22','24','26']:
    data=pd.read_excel('entgal.xlsx',sheet_name='ENTREPAÑO C-'+caliber,skiprows=[0,2])
    for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['DFONDO M'].values[i]),
        str(data['DFRENTE M'].values[i]),
        str(data['VFONDO M'].values[i]),
        str(data['VLARGO M'].values[i]),
        str(data['SKU'].values[i]),
        caliber,
        str(data['KG / M2'].values[i]),
        str(data['PESO '].values[i]),
        str(data['M2 '].values[i]),
        str(data['CAPACIDAD DE CARGA (KG)'].values[i]),
        str(data['CON 1 REFUERZO'].values[i]),
        str(data['CON 2 REFUERZOS'].values[i]),
        'GALVANIZADO'
                  ]))
cnx.commit()

In [28]:
for i in data.columns:


    
    print("str(data['"+i+"'].values[i]),")

str(data['DFONDO M'].values[i]),
str(data['DFRENTE M'].values[i]),
str(data['VFONDO M'].values[i]),
str(data['VLARGO M'].values[i]),
str(data['SKU'].values[i]),
str(data['CALIBRE'].values[i]),
str(data['KG / M2'].values[i]),
str(data['PESO '].values[i]),
str(data['M2 '].values[i]),
str(data['PRECIO VENTA'].values[i]),
str(data['CAPACIDAD DE CARGA (KG)'].values[i]),
str(data['CON 1 REFUERZO'].values[i]),
str(data['CON 2 REFUERZOS'].values[i]),


In [61]:
cursor.execute('truncate table estanteria_entrepanios')
cursor.execute('alter table estanteria_entrepanios AUTO_INCREMENT = 1')

In [53]:
cnx.commit()

In [66]:
e=pd.read_sql('select * from estanteria_entrepanios',cnx)

C:\Users\hp\AppData\Local\Temp\ipykernel_6452\401490957.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  e=pd.read_sql('select * from estanteria_entrepanios',cnx)


In [68]:
e['type'].unique()

array(['PINTADO', 'GALVANIZADO'], dtype=object)

In [11]:
data=pd.read_excel('angulos.xlsx',sheet_name='ANGULO RANURADO C-14',skiprows=[0,1,2,])

In [7]:
query='INSERT INTO gangplank_angles(length,sku,caliber,weight,m2,deep) values(%s,%s,%s,%s,%s,%s)'

In [10]:
for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['Unnamed: 0'].values[i]),
        str(data['SKU'].values[i]),
        '14',
        str(data['PESO (KG).1'].values[i]),
        str(data['M2.1'].values[i]),
        '0.76'
                  ]))
cnx.commit()

In [4]:
data.columns

Index(['Unnamed: 0', 'SKU', 'PESO (KG)', 'M2', 'PRECIO', 'SKU.1',
       'PESO (KG).1', 'M2.1', 'PRECIO.1'],
      dtype='object')

In [12]:
data

,Unnamed: 0,SKU,PESO (KG),M2,PRECIO,SKU.1,PESO (KG).1,M2.1,PRECIO.1
0,0.01,TC0000123991,0.01098,0.00144,1.540667,TC0000124357,0.01403,0.00184,1.968631
1,0.02,TC0000123992,0.02196,0.00288,2.981937,TC0000124358,0.02806,0.00368,3.810253
2,0.03,TC0000123993,0.03294,0.00432,4.472905,TC0000124359,0.04209,0.00552,5.715379
3,0.04,TC0000123994,0.04392,0.00576,5.963874,TC0000124360,0.05612,0.00736,7.620505
4,0.05,TC0000123995,0.05490,0.00720,7.454842,TC0000124361,0.07015,0.00920,9.525632
...,...,...,...,...,...,...,...,...,...
361,3.62,TC0000124352,3.97476,0.52128,539.730568,TC0000124718,5.07886,0.66608,689.655726
362,3.63,TC0000124353,3.98574,0.52272,541.221537,TC0000124719,5.09289,0.66792,691.560853
363,3.64,TC0000124354,3.99672,0.52416,542.712505,TC0000124720,5.10692,0.66976,693.465979
364,3.65,TC0000124355,4.00770,0.52560,544.203474,TC0000124721,5.12095,0.67160,695.371105


In [14]:
data=pd.read_excel('respaldos.xlsx',skiprows=[0,2],sheet_name='RESPALDOS C-24')

In [19]:
query='INSERT INTO respaldos(deep,front,sku,caliber,kgm2,weight,m2) values(%s,%s,%s,%s,%s,%s,%s)'

In [22]:
for i in range(len(data)):
        cursor.execute(query,tuple([
        str(data['DESARROLLO'].values[i]),
        str(data['Unnamed: 2'].values[i]),
        str(data['SKU'].values[i]),
        str(data['CALIBRE'].values[i]),
        str(data['KG / M2'].values[i]),
        str(data['PESO '].values[i]),
        str(data['M2 '].values[i]),
                  ]))
cnx.commit()

In [16]:
data.columns

Index(['Unnamed: 0', 'DESARROLLO', 'Unnamed: 2', 'SKU', 'CALIBRE', 'KG / M2',
       'PESO ', 'M2 ', 'PRECIO VENTA'],
      dtype='object')